# Enriched Approach

In [20]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.decomposition import PCA
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV

## Cleaning the data

In [3]:
data_set = pd.read_csv("Full_data.csv")
original_size = len(data_set)

### Dropping columns

In [4]:
drop_columns = ["Unnamed: 0", "lyrics", "title", "duration_ms", "data", "analysis_url", "id", "track_href", "type", "uri"]
data_set = data_set.drop(drop_columns, axis=1)
data_set["style"], data_set["valence"]= data_set["valence"], data_set["style"]
data_set = data_set.rename(columns={"style": "valence", "valence": "style"})

### Dropping invalid songs

In [5]:
null_elem = data_set.isnull().any(axis=1)
data_set = data_set[~null_elem]
data_set = data_set.reset_index(drop=True)
reduced_size = len(data_set)

### Encoding Styles

In [6]:
# Encoding author, style and title
to_encode = ["style", "author",]
new_col_names = ["code_style", "code_author"]
new_col_index = [len(data_set.columns)-1, 1]
encoders = []
for i in range(len(to_encode)):
    values = data_set[to_encode[i]]
    encoder = LabelEncoder()
    encoders.append(encoder)
    new_col = encoder.fit_transform(values) 
    data_set.insert(new_col_index[i], new_col_names[i], new_col)
    
# Dropping author, style and title
drop_columns = ["style", "author"]
data_set = data_set.drop(drop_columns, axis=1)

### Standardization

In [7]:
cols = data_set.columns
cols_to_std = cols.drop(["code_style"])
data_set[cols_to_std] = data_set[cols_to_std].apply(lambda x: (x-x.mean())/x.std())

### Final data set

In [8]:
reduction = (original_size-reduced_size)/original_size
print("Data set reduction = ", reduction)
data_set

Data set reduction =  0.286112349531877


,code_author,valence,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,code_style
0,0.545075,-1.601681,-0.302035,-0.262485,-0.076129,-0.327603,-0.355787,-0.547960,0.416045,0.705209,-0.388518,0.266144,0.266299,9
1,-0.710051,-1.450240,-0.650304,0.250378,-0.660477,-0.327603,0.777248,-0.580874,0.506416,0.705209,-0.624666,-0.901248,-2.238318,9
2,0.170012,0.906803,0.037068,0.756987,0.129627,-0.327603,-0.072528,0.839138,0.130039,0.705209,1.000506,-1.340523,0.266299,6
3,-1.151464,0.114650,0.923015,1.376176,-0.014402,-0.298270,-0.355787,-0.209414,-0.303892,-1.417976,-0.398785,-0.398749,0.266299,6
4,-1.151464,0.638869,0.379227,-0.969236,0.923845,-0.327603,-0.922305,3.467569,0.523329,0.705209,1.029841,1.682770,0.266299,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32020,-0.525744,-0.522179,-0.975966,-1.607187,1.281861,-0.327603,-0.922305,1.055431,1.199595,0.705209,0.391800,1.951085,0.266299,15
32021,-1.120132,0.436948,-1.147259,-1.275703,1.425890,-0.327603,-1.205564,-0.458621,1.331113,-1.417976,0.208454,1.920414,0.266299,15
32022,0.980038,-0.902723,-1.081974,0.231615,1.211904,-0.327603,-0.355787,1.873583,1.365696,-1.417976,1.161850,0.379457,0.266299,15
32023,1.259262,-0.793996,-1.147650,-1.169377,1.450581,-0.327595,0.777248,-0.634947,1.492165,0.705209,2.276588,1.450467,0.266299,15


### PCA analysis

data = data_set.drop(["code_style"], axis=1)
pca = PCA()
comp = pca.fit(data)
importance = comp.explained_variance_/sum(comp.explained_variance_)
for i in importance:
    print("{0:.3f}".format(i), end=" ")

## MLP

### Spliting the data set

In [9]:
Y = data_set["code_style"]
X = data_set.drop(["code_style"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

### Simple Test

In [16]:
clf = MLPClassifier(solver="sgd", alpha=0.001, activation="logistic", hidden_layer_sizes=(16+1, 50))

clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.03      0.10      0.05       271
           1       0.00      0.00      0.00       240
           2       0.00      0.00      0.00       172
           3       0.00      0.00      0.00       226
           4       0.00      0.00      0.00       255
           5       0.00      0.00      0.00       197
           6       0.00      0.00      0.00       245
           7       0.00      0.00      0.00       232
           8       0.00      0.00      0.00       197
           9       0.00      0.00      0.00       230
          10       0.00      0.00      0.00       228
          11       0.02      0.00      0.01       202
          12       0.03      0.10      0.05       274
          13       0.00      0.00      0.00       137
          14       0.00      0.00      0.00       229
          15       0.05      0.95      0.09       233
          16       0.00      0.00      0.00       147
          17       0.05    

/home/tristan/.local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/tristan/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Grid Approach

In [11]:
mlp = MLPClassifier(max_iter=100)

In [12]:
parameter_space = {
    "hidden_layer_sizes": [(100), (25,25,25,25), (98, 11, 16)],
    "activation" : ["tanh", "logistic"],
    "solver" : ["sgd", "adam"],
    "alpha" : [0.0001, 0.05, 0.1, 1],
    "learning_rate" : ["invscaling", "adaptive"]
}

In [13]:
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

/home/tristan/.local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=100,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_sta...
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['tanh', 'logistic'],
                         'alpha': [0.0001, 0.05, 0.1, 1],
                         'hidden

In [14]:
clf.best_params_

{'activation': 'tanh',
 'alpha': 0.1,
 'hidden_layer_sizes': 100,
 'learning_rate': 'invscaling',
 'solver': 'adam'}

### Grid best result

In [15]:
new = MLPClassifier(solver="adam", alpha=0.1, activation="tanh", hidden_layer_sizes=(100), learning_rate="invscaling")

new.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.15      0.07      0.10       271
           1       0.13      0.06      0.08       240
           2       0.28      0.40      0.33       172
           3       0.36      0.43      0.39       226
           4       0.18      0.19      0.18       255
           5       0.18      0.16      0.17       197
           6       0.24      0.17      0.20       245
           7       0.32      0.26      0.29       232
           8       0.22      0.24      0.23       197
           9       0.17      0.10      0.12       230
          10       0.20      0.29      0.24       228
          11       0.34      0.46      0.39       202
          12       0.16      0.19      0.18       274
          13       0.52      0.69      0.59       137
          14       0.29      0.38      0.33       229
          15       0.32      0.49      0.39       233
          16       0.28      0.24      0.26       147
          17       0.12    

/home/tristan/.local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Random Approach